In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))


In [2]:
import sys
import os
import numpy as np

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/xgboost/xgboost4j/jars/xgboost4j_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/xgboost4j-spark_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/spark-arrow-datasource-standard-1.1.0-jar-with-dependencies.jar pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/xgboost/xgboost4j/jars/xgboost4j_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/xgboost4j-spark_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/spark-arrow-datasource-0.9.0-jar-with-dependencies.jar pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/xgboost/jars-for-aws-cpu/1.3/xgboost4j_2.12-1.3.3.jar,/home/xgboost/jars-for-aws-cpu/1.3/xgboost4j-spark_2.12-1.3.3.jar pyspark-shell'
# os.environ['ARROW_LIBHDFS3_DIR'] = '/home/xgboost/miniconda3/lib'
os.environ['LD_LIBRARY_PATH'] = '/home/xgboost/miniconda3/lib'

In [3]:
SPARK_HOME='/home/xgboost/spark-3.0.0-bin-hadoop2.7'
import findspark
findspark.init(SPARK_HOME)

In [4]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time, sleep
import subprocess
import math

In [5]:
clients = ['sr572']
nrepeat = 1
executors_per_node = 1
nodes=len(clients)
cores_per_executor=8
task_per_core=1

cache_size=30000
arrow_datasource_jar = '/home/xgboost/xgboost4j/jars/spark-arrow-datasource-standard-1.1.0-jar-with-dependencies.jar'
#arrow_datasource_jar = '/home/xgboost/xgboost4j/jars/spark-arrow-datasource-0.9.0-jar-with-dependencies.jar'

def start_cntx():
    conf = SparkConf()\
        .set('spark.default.parallelism', '{:d}'.format(nodes*executors_per_node*cores_per_executor))\
        .set('spark.executor.instances', '{:d}'.format(executors_per_node*nodes))\
        .set('spark.rdd.compress', 'False')\
        .set('spark.files.maxPartitionBytes', '512m')\
        .set('spark.executor.cores','{:d}'.format(cores_per_executor))\
        .set('spark.executor.memory', '20g') \
        .set('spark.executor.memoryOverhead', '4g') \
        .set('spark.task.cpus', '{:d}'.format(task_per_core))\
        .set('spark.driver.memory', '10g')\
        .set('spark.executor.extraJavaOptions',\
            '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails')\
        .set('spark.driver.maxResultSize', '0')\
        .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')\
        .set('spark.memory.offHeap.enabled','True')\
        .set('spark.memory.offHeap.size','10g')\
        .set('spark.executorEnv.LD_LIBRARY_PATH', '/home/xgboost/miniconda3/lib')\
        .set('spark.driver.extraClassPath', arrow_datasource_jar) \
        .set('spark.executor.extraClassPath', arrow_datasource_jar) \
        .setAppName('mortgage')

    spark = SparkSession.builder\
                .master('spark://sr572:7077')\
                .config(conf=conf)\
                .getOrCreate()
        
    sc = spark.sparkContext
    sc.setLogLevel('INFO')
    sc.addPyFile('/home/xgboost/xgboost4j/sparkxgb_1.24.zip')
    return sc, spark    

In [6]:
SPARK_HOME='/home/xgboost/spark-3.0.0-bin-hadoop2.7'
!$SPARK_HOME/sbin/start-master.sh
!$SPARK_HOME/sbin/start-slave.sh spark://sr572:7077 -c 8

/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
starting org.apache.spark.deploy.master.Master, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.master.Master-1-sr572.out
/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
starting org.apache.spark.deploy.worker.Worker, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.worker.Worker-1-sr572.out


In [7]:
sc, spark = start_cntx()
appid = sc.applicationId
        
test_data = spark.read.format('arrow').load('file:///home/xgboost/data/HiBench10Kx50.dataframe.float.parquet')

from sparkxgb import XGBoostClassificationModel
model = XGBoostClassificationModel().load('file:///home/xgboost/model')
model.transform(test_data).select('label', 'rawPrediction', 'probability', 'prediction').show(5)
       


******** XGBoostClassificationModel ********
+-----+--------------------+--------------------+----------+
|label|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|  1.0|[-0.8339383602142...|[0.16606163978576...|       1.0|
|  1.0|[-0.8501297831535...|[0.14987021684646...|       1.0|
|  0.0|[-0.1639787405729...|[0.83602125942707...|       0.0|
|  1.0|[-0.9957962036132...|[0.00420379638671...|       1.0|
|  1.0|[-0.8613203763961...|[0.13867962360382...|       1.0|
+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [8]:
sc.stop()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/xgboost/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/xgboost/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/xgboost/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
/home/xgboost/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashe

In [9]:
!$SPARK_HOME/sbin/stop-slave.sh
!$SPARK_HOME/sbin/stop-master.sh

/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
stopping org.apache.spark.deploy.worker.Worker
/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
stopping org.apache.spark.deploy.master.Master
